## 코딩테스트

### MNIST 손글씨 숫자 분류

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 데이터 로딩 및 전처리

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'CPU/GPU: {device}')

transform = transforms.ToTensor()

train_set = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)

test_set = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

CPU/GPU: cuda


100%|██████████| 9.91M/9.91M [00:06<00:00, 1.60MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 156kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.27MB/s]


<img src ='./코딩테스트_데이터준비.png' width=500>

### CNN 모델 선언

In [14]:
# MNIST 데이터셋의 기본 구성
# 학습 데이터 (train_set): 60,000개
# 테스트 데이터 (test_set): 10,000개

print(f"학습 데이터의 총 개수: {len(train_set)}")

학습 데이터의 총 개수: 60000


In [10]:
image, label = train_set[0]

# 이미지 shape 확인
print(f"이미지 shape: {image.shape}")  # torch.Size([1, 28, 28])
print(f"라벨: {label}")

이미지 shape: torch.Size([1, 28, 28])
라벨: 5


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)   #[1,28,28] ->[16,26,26] 커널사이즈 3*3 , 스트라이드 기본값 1
        self.pool = nn.MaxPool2d(2, 2)                  # 맥스풀링하면 절반됨 [16,26,26] ->[16,13,13]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)   #[16,13,13] ->[32,11,11] 커널사이즈 3*3 , 스트라이드 기본값 1
                                                        # 맥스풀링하면 절반됨 [32,11,11] ->[32,5,5]  
                                                        # 마지막 행과 열은 2x2 풀링 윈도우에 들어갈 수 없어서 그냥 버려지는 게 맞습니다.
        self.fc1 = nn.Linear(32 * 5 * 5, 128)           # 128개의 특징(벡터)으로 압축 /이 128은 하이퍼파라미터예요!
        self.fc2 = nn.Linear(128, 10)                   #  128차원의 특징 벡터를 MNIST 숫자 분류 문제(클래스 10개)에 맞게



    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # 문제. 아래의 주석을 풀고 알맞은 Flatten 내용을 추가하시오.
        x = x.view(-1,32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
model = CNN()

### 손실함수와 옵티마이저

In [5]:
#CrossEntropyLoss는 다중 클래스 분류 문제에서 자주 쓰이는 손실 함수로, 예측값과 실제값 사이의 오차를 측정합니다.
criterion = nn.CrossEntropyLoss()


# 문제: 아래의 주석을 해제, Adam 옵티마이저를 설정하고 학습률은 0.001로 지정하세요.
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

### 학습

In [6]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        # 문제: 아래의 주석 해제, 손실(loss)를 계산하세요.
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {running_loss / len(train_loader):.4f}")

[Epoch 1] Loss: 0.2085
[Epoch 2] Loss: 0.0636
[Epoch 3] Loss: 0.0449
[Epoch 4] Loss: 0.0335
[Epoch 5] Loss: 0.0266


- 학습 데이터:
    - MNIST의 학습 데이터는 60,000개의 이미지입니다.
- 배치 크기:
    - 배치 크기가 64이므로, 한 번에 64개의 이미지를 처리합니다.
- 배치 개수:
    - 학습 데이터 60,000개를 배치 크기 64로 나누면, 총 배치 개수는 938개입니다.
    - 즉, 한 에폭 동안 모델은 938번의 배치에 대해 학습을 하게 됩니다.
- running_loss:
    - 각 배치에 대해 손실 값을 계산한 후, 그 손실 값은 running_loss에 누적됩니다.
    - 이 손실 값들은 해당 938개의 배치에서 계산된 손실들의 합입니다.
- 평균 손실 계산:
    - running_loss / len(train_loader)에서 len(train_loader)는 배치 개수, 즉 938입니다.
    - running_loss는 938개의 배치에 대한 손실의 합이므로, 이를 938개 배치의 평균 손실로 나누게 되는 것입니다.

### 평가 및 저장

In [7]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)   # 각 배치에서 가장 높은 확률을 가진 클래스를 예측값으로 선택합니다.
        total += labels.size(0)                # 전체 테스트 데이터에 대한 정확도를 계산하려면, 전체 데이터 수를 누적해야 합니다.
        # 문제: 주석해제 후, 정확히 예측한 개수를 누적하세요.
        correct +=  (predicted == labels).sum().item()

print(f'테스트 정확도: {100 * correct / total:.2f}%')

# 문제: 모델 저장 (선택) - 학습된 모델을 'mnist_cnn.pth' 파일로 저장하는 코드를 작성하시오
torch.save(model, 'mnist_cnn.pth')

테스트 정확도: 98.77%


#### 위의 내용을 CUDA에서 동작하도록 하나의 셀에 작성 완성하시오

In [15]:
# 문제: 전체 소스 작성

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 데이터 준비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'CPU/GPU: {device}')

transform = transforms.ToTensor()

train_set = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)

test_set = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)


# 모델
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)   #[1,28,28] ->[16,26,26] 커널사이즈 3*3 , 스트라이드 기본값 1
        self.pool = nn.MaxPool2d(2, 2)                  # 맥스풀링하면 절반됨 [16,26,26] ->[16,13,13]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)   #[16,13,13] ->[32,11,11] 커널사이즈 3*3 , 스트라이드 기본값 1
                                                        # 맥스풀링하면 절반됨 [32,11,11] ->[32,5,5]  
                                                        # 마지막 행과 열은 2x2 풀링 윈도우에 들어갈 수 없어서 그냥 버려지는 게 맞습니다.
        self.fc1 = nn.Linear(32 * 5 * 5, 128)           # 128개의 특징(벡터)으로 압축 /이 128은 하이퍼파라미터예요!
        self.fc2 = nn.Linear(128, 10)                   #  128차원의 특징 벡터를 MNIST 숫자 분류 문제(클래스 10개)에 맞게



    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

# 학습
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {running_loss / len(train_loader):.4f}")

# 검증
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1) 
        total += labels.size(0)               
        correct +=  (predicted == labels).sum().item()

print(f'테스트 정확도: {100 * correct / total:.2f}%')

# 모델 저장
torch.save(model, 'mnist_cnn.pth')

CPU/GPU: cuda
[Epoch 1] Loss: 0.2461
[Epoch 2] Loss: 0.0662
[Epoch 3] Loss: 0.0468
[Epoch 4] Loss: 0.0350
[Epoch 5] Loss: 0.0299
테스트 정확도: 98.95%
